In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Embedding,LSTM
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import re

In [3]:
from google.colab import files
uploaded=files.upload()

Saving Tweets.csv to Tweets.csv


In [4]:
dataset=pd.read_csv("Tweets.csv")

In [5]:
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,latitude,longitude
0,5.703061e+17,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,35.888455,-119.273781
1,5.703011e+17,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,37.770971,-119.941025
2,5.703011e+17,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,35.684863,-119.709299
3,5.703010e+17,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,37.061159,-119.279135
4,5.703008e+17,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,36.790587,-120.867752


In [6]:
dataset.tail()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,latitude,longitude
14635,5.695877e+17,positive,0.3487,NaN,0.0000,American,KristenReenders,0,@AmericanAir thank you we got on a different f...,2015-02-22 12:01:01 -0800,40.7473,-73.9721
14636,5.695874e+17,negative,1.0000,Customer Service Issue,1.0000,American,itsropes,0,@AmericanAir leaving over 20 minutes Late Flig...,2015-02-22 11:59:46 -0800,40.7461,-73.9821
14637,5.695872e+17,neutral,1.0000,NaN,NaN,American,sanyabun,0,@AmericanAir Please bring American Airlines to...,2015-02-22 11:59:15 -0800,40.7088,-73.9210
14638,5.695872e+17,negative,1.0000,Customer Service Issue,0.6659,American,SraJackson,0,"@AmericanAir you have my money, you change my ...",2015-02-22 11:59:02 -0800,40.7179,-74.0146
14639,5.695871e+17,neutral,0.6771,NaN,0.0000,American,daviddtwu,0,@AmericanAir we have 8 ppl so we need 2 know h...,2015-02-22 11:58:51 -0800,40.7701,-73.9832


In [7]:
dataset['airline'].unique()

array(['Virgin America', 'United', 'Southwest', 'Delta', 'US Airways',
       'American'], dtype=object)

In [8]:
dataset['airline_sentiment'].unique()

array(['neutral', 'positive', 'negative'], dtype=object)

In [9]:
dataset.isnull().sum()

tweet_id                           0
airline_sentiment                  0
airline_sentiment_confidence       0
negativereason                  5462
negativereason_confidence       4118
airline                            0
name                               0
retweet_count                      0
text                               0
tweet_created                      0
latitude                           0
longitude                          0
dtype: int64

In [10]:
data=dataset[["text","airline_sentiment"]]
data.head()

,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [11]:
dataset=data[data.airline_sentiment !='neutral']
dataset.head()

,text,airline_sentiment
1,@VirginAmerica plus you've added commercials t...,positive
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative
5,@VirginAmerica seriously would pay $30 a fligh...,negative
6,"@VirginAmerica yes, nearly every time I fly VX...",positive


In [12]:
dataset.airline_sentiment.unique()

array(['positive', 'negative'], dtype=object)

In [13]:
dataset['text'][1]

"@VirginAmerica plus you've added commercials to the experience... tacky."

In [14]:
import nltk
from nltk.corpus import stopwords

In [15]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [16]:
print(stopwords.words("english"))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [17]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11541 entries, 1 to 14638
Data columns (total 2 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   text               11541 non-null  object
 1   airline_sentiment  11541 non-null  object
dtypes: object(2)
memory usage: 590.5+ KB


In [18]:
dataset['text']=dataset['text'].apply(lambda x:x.lower())

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [19]:
print(dataset['text'].iloc[3])

@virginamerica seriously would pay $30 a flight for seats that didn't have this playing.
it's really the only bad thing about flying va


In [20]:
dataset['text']=dataset['text'].apply(lambda x:re.sub('[^a-zA-Z0-9\s]','',x))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [21]:
print(dataset['text'].iloc[3])

virginamerica seriously would pay 30 a flight for seats that didnt have this playing
its really the only bad thing about flying va


In [23]:
dataset.shape

(11541, 2)

In [24]:
stop_words=set(stopwords.words("english"))

In [26]:
dataset['text']=dataset['text'].apply(lambda x:' '.join([word for word in x.split() if word not in (stop_words)]))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
print(dataset['text'].iloc[3])

virginamerica seriously would pay 30 flight seats didnt playing really bad thing flying va


In [28]:
max_features=2000
tokenizer=Tokenizer(num_words=max_features,split=' ')
tokenizer.fit_on_texts(dataset['text'].values)

In [29]:
print(dataset['text'].iloc[3])

virginamerica seriously would pay 30 flight seats didnt playing really bad thing flying va


In [30]:
seq=tokenizer.texts_to_sequences(dataset['text'].values)

In [31]:
seq[3]

[43, 283, 34, 185, 186, 2, 121, 104, 1980, 56, 115, 376, 78, 1807]

In [32]:
print(seq[3])

[43, 283, 34, 185, 186, 2, 121, 104, 1980, 56, 115, 376, 78, 1807]


In [33]:
len(seq[3])

14

In [34]:
len(seq[12])

15

In [35]:
print(seq[12])

[43, 769, 88, 1190, 710, 2, 242, 683, 47, 434, 146, 46, 383, 927, 1265]


In [36]:
seq=pad_sequences(seq)

In [37]:
print(seq[12])

[   0    0    0    0    0    0    0   43  769   88 1190  710    2  242
  683   47  434  146   46  383  927 1265]


In [38]:
print(seq[20])

[   0    0    0    0    0    0    0    0    0    0    0    0   43  296
 1073 1074 1652  321  315  165 1137  101]


In [39]:
seq.shape

(11541, 22)

In [40]:
y=pd.get_dummies(dataset['airline_sentiment'],drop_first=True).values

In [41]:
y[3]

array([0], dtype=uint8)

In [42]:
y[20]

array([0], dtype=uint8)

In [43]:
x_train,x_test,y_train,y_test=train_test_split(seq,y,test_size=0.2,random_state=0)

In [44]:
x_train.shape

(9232, 22)

In [45]:
x_train[3]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          1,   21,  698, 1485,   88,  239,    2, 1827,   46,   10, 1938],
      dtype=int32)

In [46]:
y_train[3]

array([1], dtype=uint8)

In [47]:
embed_dimension=128

In [48]:
model=Sequential()

In [49]:
model.add(Embedding(max_features,embed_dimension,input_length=seq.shape[1]))
model.add(LSTM(128,dropout=0.2,recurrent_dropout=0.2))
model.add(Dense(1,activation='sigmoid'))

In [50]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 22, 128)           256000    
_________________________________________________________________
lstm (LSTM)                  (None, 128)               131584    
_________________________________________________________________
dense (Dense)                (None, 1)                 129       
Total params: 387,713
Trainable params: 387,713
Non-trainable params: 0
_________________________________________________________________


In [51]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [52]:
history=model.fit(x_train,y_train,batch_size=100,validation_data=(x_test,y_test),epochs=10)

Epoch 1/10
93/93 [==============================] - 13s 105ms/step - loss: 0.4996 - accuracy: 0.8012 - val_loss: 0.2386 - val_accuracy: 0.9091
Epoch 2/10
93/93 [==============================] - 9s 100ms/step - loss: 0.1981 - accuracy: 0.9242 - val_loss: 0.2031 - val_accuracy: 0.9251
Epoch 3/10
93/93 [==============================] - 9s 100ms/step - loss: 0.1442 - accuracy: 0.9454 - val_loss: 0.2026 - val_accuracy: 0.9272
Epoch 4/10
93/93 [==============================] - 9s 99ms/step - loss: 0.1212 - accuracy: 0.9566 - val_loss: 0.2199 - val_accuracy: 0.9255
Epoch 5/10
93/93 [==============================] - 9s 100ms/step - loss: 0.1043 - accuracy: 0.9615 - val_loss: 0.2284 - val_accuracy: 0.9238
Epoch 6/10
93/93 [==============================] - 9s 101ms/step - loss: 0.0862 - accuracy: 0.9680 - val_loss: 0.2485 - val_accuracy: 0.9108
Epoch 7/10
93/93 [==============================] - 9s 102ms/step - loss: 0.0770 - accuracy: 0.9706 - val_loss: 0.2842 - val_accuracy: 0.9190
Epoch 

In [53]:
tweets=["virginamerica has the most incredible customer service i've ever experienced! so refreshing!"]

In [54]:
from nltk import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [55]:
tokenizer.fit_on_texts(tweets)

In [57]:
filtered_twts=[]
for word in tweets:
  if word not in stop_words:
    filtered_twts.append(word)

In [58]:
filtered_twts

["virginamerica has the most incredible customer service i've ever experienced! so refreshing!"]

In [59]:
seq1=tokenizer.texts_to_sequences(filtered_twts)

In [60]:
seq1

[[43, 1286, 11, 10, 77, 898]]

In [61]:
print(seq1)

[[43, 1286, 11, 10, 77, 898]]


In [62]:
seq1=pad_sequences(seq1,maxlen=22)

In [63]:
seq1

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,   43, 1286,   11,   10,   77,  898]],
      dtype=int32)

In [64]:
y_pred=model.predict_classes(seq1)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:450: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [65]:
y_pred

array([[0]], dtype=int32)